# Question Answering - Implantação

Este é um componente que utiliza a biblioteca [simpletransformers](https://simpletransformers.ai/) para implantar o modelo Bert de Q&A. Os pesos utilizados para o modelo foram obtidos através do treino com a base de dados aberta [squad](https://github.com/nunorc/squad-v1.1-pt) na língua portuguesa.

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class Model(object):
    
    
    def __init__(self):
    
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/qea.joblib")
        self.model = artifacts["model"]
        self.context = artifacts["context"]
        self.question = artifacts["question"]
        
    
    def format_input(self, df):
        
        contexts = np.array(df[self.context])
        questions = np.array(df[self.question])
        
        data = []

        for i in range(len(contexts)):

            data.append(
                {
                 'context': contexts[0],
                 'qas': [{'id': i,
                         'question': questions[0]}]
                }
            )
        
        return data


    def predict(self, X, feature_names, meta=None):
        
        df = pd.DataFrame(X)
        
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)
            
        X = self.format_input(df)
            
        # Realiza a predição
        y_pred = self.model.predict(X)

        return y_pred

## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>